In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

print("Training Data Sample:")
print(train_data.head())

print("\nTest Data Sample:")
print(test_data.head())

In [ ]:
from sklearn.preprocessing import LabelEncoder

def preprocess_data(data, is_train=True):
    
    data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)  
    if 'Embarked' in data.columns:
        data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

    label_enc = LabelEncoder()
    data['Sex'] = label_enc.fit_transform(data['Sex'])
    if 'Embarked' in data.columns:
        data['Embarked'] = label_enc.fit_transform(data['Embarked'])

    if is_train:
        X = data.drop('Survived', axis=1)
        y = data['Survived']
        return X, y
    return data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = preprocess_data(train_data)

X_test_final = preprocess_data(test_data, is_train=False)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_final = scaler.transform(X_test_final)

In [ ]:
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(f"\n{name}")
    print("Accuracy:", accuracy_score(y_val, y_pred))
    print(classification_report(y_val, y_pred))

In [ ]:
best_model = classifiers["Random Forest"]

test_predictions = best_model.predict(X_test_final)

output = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})

print("Predictions for the test set:")
print(output.head())